# Beaver Tutorial 5: GWAS Analysis (Data Owner)

Run a **GWAS pipeline** with step-by-step approval.

You control each step:
1. **Merge** - Combine datasets
2. **PCA** - Population stratification  
3. **Association** - Logistic regression
4. **Plots** - Manhattan & QQ plots

---

## Step 1: Setup

In [1]:
# import beaver
# beaver.dev()

In [2]:
!uv pip install pandas numpy matplotlib -q
print("Dependencies installed!")

Dependencies installed!


In [3]:
import beaver
from beaver import Twin

bv = beaver.ctx()
session = bv.active_session()

# Clean up old requests from previous runs
deleted = session.reset(force=True)
print(f"🧹 Cleaned {deleted} old files")

print(f"You: {bv.user}")
print(f"Peer: {session.peer}")

🔄 Auto-load replies enabled for client1@sandbox.local (polling every 0.5s)
🟢 Active session loaded: d9efe11cc2d0
   Peer: client2@sandbox.local
  Deleted: b56bc99ec25c4fc4be6978405a461d90.beaver
  Deleted: remote_vars.json
  Deleted: data/ (0 files)
  Deleted: inbox/ (1 files)
✓ Session reset: 3 files deleted
🧹 Cleaned 3 old files
You: client1@sandbox.local
Peer: client2@sandbox.local


In [4]:
import shutil
plink = shutil.which("plink")
if plink:
    print(f"PLINK found: {plink}")
else:
    print("PLINK not found! Install from: https://www.cog-genomics.org/plink/")

PLINK found: /Users/madhavajay/micromamba/bin/plink


## Step 2: Load & Publish Data

In [5]:
from pathlib import Path

# Path to genomic data
DATA_DIR = Path("/Users/madhavajay/dev/biovaults/datasets/jordan_gwas")

# Real datasets (without extension)
REAL_DATASET1 = str(DATA_DIR / "Circassian_qc")
REAL_DATASET2 = str(DATA_DIR / "Chechen_qc")
MOCK_DATASET = str(DATA_DIR / "mock")

print(f"Real: {REAL_DATASET1}")
print(f"Mock: {MOCK_DATASET}")

Real: /Users/madhavajay/dev/biovaults/datasets/jordan_gwas/Circassian_qc
Mock: /Users/madhavajay/dev/biovaults/datasets/jordan_gwas/mock


In [6]:
# Create GWAS data Twin
gwas_data = Twin(
    public={
        "dataset1_prefix": MOCK_DATASET,
        "dataset2_prefix": MOCK_DATASET,
        "dataset1_name": "mock",
        "dataset2_name": "mock",
        "n_samples_1": 30,
        "n_variants_1": 100,
        "n_samples_2": 30,
        "n_variants_2": 100,
    },
    private={
        "dataset1_prefix": REAL_DATASET1,
        "dataset2_prefix": REAL_DATASET2,
        "dataset1_name": "Circassian_qc",
        "dataset2_name": "Chechen_qc",
        "n_samples_1": 139,
        "n_variants_1": 642505,
        "n_samples_2": 145,
        "n_variants_2": 641285,
    },
    name="gwas_data",
)

session.remote_vars["gwas_data"] = gwas_data
print("GWAS data published!")

🌍 Using PUBLIC data from Twin 'gwas_data...'
📢 Published Twin 'gwas_data' (public side available at: /Users/madhavajay/dev/biovault-desktop/workspace2/biovault/sandbox/client1@sandbox.local/datasites/client1@sandbox.local/shared/biovault/sessions/d9efe11cc2d0/data/6ee1ce2536f14c599352ad0f94e18015.beaver)
GWAS data published!


---
## GWAS Step 1: Merge Datasets

**Run DS notebook to send merge request, then return here.**

In [7]:
merge_request = bv.wait_for_request(gwas_data, timeout=300)
if merge_request:
    print(f"Merge request from {merge_request.sender}")
    display(merge_request)

⏳ Waiting for request on 'gwas_data'...
📬 Request received: request_gwas_step1_merge_for_merge_result
   From: client2@sandbox.local
Merge request from client2@sandbox.local


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
⚡ Action: gwas_step1_merge_result
   Request from: client2@sandbox.local
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📋 Function: gwas_step1_merge

⚙️  Static Parameters:
  • arg[0]: dict = {'_beaver_twin_ref': True, 'twin_id': '684ad36d...

💡 Next Steps:
   .run_mock()   → Test on mock/public data (safe preview)
   .run()        → Execute on real/private data
   .run_both()   → Run on both mock & real for comparison
   .reject(msg)  → Decline this request

🆔 IDs: comp=51adb04d0849... result=5859fc5d6659...
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
session.inbox()

name,id,sender,type,size_bytes,created_at,reply_to
request_gwas_step1_merge_for_merge_result,cc8fcfca688b...,client2@sandbox.local,ComputationRequest,3503,2025-12-10 14:40:51,


In [9]:
# session.open()

In [ ]:
# Run merge on both mock and real data
# We need both to create the pca_input Twin for DS
if merge_request:
    merge_result = merge_request.run_both()
    
    r = merge_result.data.private
    print(f"\nReal: {r['n_samples']} samples, {r['n_variants']} variants")
    print(f"Cases: {r['cases']}, Controls: {r['controls']}")
    
    m = merge_result.data.public
    print(f"\nMock: {m['n_samples']} samples, {m['n_variants']} variants")

In [ ]:
# Approve merge results (sends summary to DS)
if merge_result:
    merge_result.approve()
    print("Merge summary sent to DS!")
    
    # Publish pca_input Twin for the next step
    # Public: mock merge result paths (for DS to run locally)
    # Private: real merge result paths (for DO to run)
    merge_pub = merge_result.data.public
    merge_priv = merge_result.data.private
    
    pca_input = Twin(
        public=merge_pub,   # Mock paths from DS's merge
        private=merge_priv,  # Real paths from DO's merge
        name="pca_input"
    )
    session.remote_vars["pca_input"] = pca_input
    print("✓ Published pca_input for next step")

---
## GWAS Step 2: PCA

**DS will send PCA request after receiving merge summary.**

In [ ]:
# session.inbox()

In [ ]:
# Wait for PCA request on pca_input
pca_request = bv.wait_for_request(pca_input, timeout=300)
if pca_request:
    print(f"PCA request from {pca_request.sender}")
    display(pca_request)

In [ ]:
# Run PCA on both mock and real data
if pca_request:
    pca_result = pca_request.run_both()
    
    r = pca_result.data.private
    print(f"\nReal PCA: {r['n_pcs']} PCs for {r['n_samples']} samples")
    
    # Show PCA plot from real data
    pca_result.data.show_figures("private")

In [ ]:
# Approve PCA results (sends summary + plot to DS)
if pca_result:
    pca_result.approve()
    print("PCA results sent to DS!")
    
    # Publish assoc_input Twin for the next step
    pca_pub = pca_result.data.public
    pca_priv = pca_result.data.private
    
    assoc_input = Twin(
        public=pca_pub,   # Mock paths
        private=pca_priv,  # Real paths
        name="assoc_input"
    )
    session.remote_vars["assoc_input"] = assoc_input
    print("✓ Published assoc_input for next step")

---
## GWAS Step 3: Association Testing

**DS will send association request after receiving PCA results.**

In [ ]:
# Wait for association request on assoc_input
assoc_request = bv.wait_for_request(assoc_input, timeout=600)
if assoc_request:
    print(f"Association request from {assoc_request.sender}")
    display(assoc_request)

In [ ]:
# Run association on both mock and real data
if assoc_request:
    assoc_result = assoc_request.run_both()
    
    r = assoc_result.data.private
    print(f"\nReal association complete!")
    print(f"  Tested: {r['tested_snps']} SNPs")
    print(f"  GW significant: {r['gw_significant']}")
    print(f"  Suggestive: {r['suggestive']}")
    print(f"  Lambda: {r['lambda_gc']:.3f}")
    
    if r['top_snps']:
        print(f"\nTop SNPs:")
        for snp in r['top_snps'][:5]:
            print(f"  {snp['snp']}: P={snp['p']:.2e}")

In [ ]:
# Approve association results (sends summary to DS)
if assoc_result:
    assoc_result.approve()
    print("Association results sent to DS!")
    
    # Publish plots_input Twin for the next step
    assoc_pub = assoc_result.data.public
    assoc_priv = assoc_result.data.private
    
    plots_input = Twin(
        public=assoc_pub,   # Mock paths
        private=assoc_priv,  # Real paths
        name="plots_input"
    )
    session.remote_vars["plots_input"] = plots_input
    print("✓ Published plots_input for next step")

---
## GWAS Step 4: Generate Plots

**DS will send plots request after receiving association results.**

In [ ]:
# Wait for plots request on plots_input
plots_request = bv.wait_for_request(plots_input, timeout=300)
if plots_request:
    print(f"Plots request from {plots_request.sender}")
    display(plots_request)

In [ ]:
# Generate plots on both mock and real data
if plots_request:
    plots_result = plots_request.run_both()
    
    print("\nGWAS Plots from REAL data:")
    plots_result.data.show_figures("private")

In [ ]:
# Approve plots (sends Manhattan & QQ to DS)
if plots_result:
    plots_result.approve()
    print("GWAS plots sent to DS!")

---
## Summary

You ran GWAS in 4 controlled steps:

1. **Merge** - Combined real datasets, shared counts
2. **PCA** - Computed 10 PCs, shared PCA plot
3. **Association** - Logistic regression, shared top SNPs
4. **Plots** - Generated Manhattan & QQ, shared plots

### Privacy Preserved!

- DS received **summary statistics** and **plots**
- DS did NOT receive raw genotype data
- You approved each step individually